# Missing value imputation: DropMissingData

Deletes rows with missing values.

DropMissingData works both with numerical and categorical variables.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from feature_engine.imputation import DropMissingData

# Download the data from Kaggle and store it
# in the same folder as this notebook.

data = pd.read_csv(r'C:\Users\devanshu.tayal\Downloads\houseprices.csv')

data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [2]:
# Separate the data into train and test sets.

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['Id', 'SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0,
)

X_train.shape, X_test.shape

((2043, 79), (876, 79))

## Drop data based on specific variables.

We can drop observations that show NA in any of a subset of variables.

In [3]:
# Drop data when there are NA in any of the indicated variables

imputer = DropMissingData(
    variables=['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea'],
    missing_only=False,
)

imputer.fit(X_train)

DropMissingData(missing_only=False,
                variables=['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea'])

In [4]:
# variables from which observations with NA will be deleted

imputer.variables_

['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea']

In [5]:
# Number of observations with NA before the transformation

X_train[imputer.variables].isna().sum()

Alley          1910
MasVnrType     1232
LotFrontage     333
MasVnrArea       14
dtype: int64

In [6]:
# After the transformation the rows with NA values are 
# deleted form the dataframe

train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)

# Number of observations with NA after transformation

train_t[imputer.variables].isna().sum()

Alley          0
MasVnrType     0
LotFrontage    0
MasVnrArea     0
dtype: int64

In [7]:
# Shape of dataframe before transformation

X_train.shape

(2043, 79)

In [8]:
# Shape of dataframe after transformation

train_t.shape

(19, 79)

In [9]:
# The "return_na_data()" method, returns a dataframe that contains
# the observations with NA. 

# That is, the portion of the data that is dropped when
# we apply the transform() method.

tmp = imputer.return_na_data(X_train)

tmp.shape

(2024, 79)

In [10]:
# total obs - obs with NA = final dataframe shape
#  after the transformation

1022-963

59

Sometimes, it is useful to retain the observation with NA in the production environment, to log which
observations are not being scored by the model for example.

## Drop data when variables contain %  of NA

We can drop observations if they contain less than a required percentage of values in a subset of observations.

In [11]:
# Drop data if an observation contains NA in 
# 2 of the 4 indicated variables (50%).

imputer = DropMissingData(
    variables=['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea'],
    missing_only=False,
    threshold=0.5,
)

imputer.fit(X_train)

DropMissingData(missing_only=False, threshold=0.5,
                variables=['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea'])

In [12]:
# After the transformation the rows with NA values are 
# deleted form the dataframe

train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)

# Number of observations with NA after transformation

train_t[imputer.variables].isna().sum()

Alley          1731
MasVnrType     1053
LotFrontage     166
MasVnrArea        1
dtype: int64

We see that not all missing observations were dropped, because we required the observation to have NA in more than 1 of the variables at the time. 

## Automatically select all variables

We can drop obserations if they show NA in any variable in the dataset.

When the parameter `variables` is left to None and the parameter `missing_only` is left to True, the imputer will evaluate observations based of all variables with missing data.

When the parameter `variables` is left to None and the parameter `missing_only` is switched to False, the imputer will evaluate observations based of all variables.

It is good practice to use `missing_only=True` when we set `variables=None`, so that the transformer handles the imputation automatically in a meaningful way.

### Automatically find variables with NA

In [13]:
# Find variables with NA

imputer = DropMissingData(missing_only=True)

imputer.fit(X_train)

DropMissingData()

In [14]:
# variables with NA in the train set

imputer.variables_

['MSZoning',
 'LotFrontage',
 'Alley',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [15]:
# Number of observations with NA

X_train[imputer.variables_].isna().sum()

MSZoning           2
LotFrontage      333
Alley           1910
Utilities          1
Exterior1st        1
Exterior2nd        1
MasVnrType      1232
MasVnrArea        14
BsmtQual          57
BsmtCond          58
BsmtExposure      57
BsmtFinType1      55
BsmtFinSF1         1
BsmtFinType2      55
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
Functional         1
FireplaceQu     1000
GarageType       117
GarageYrBlt      119
GarageFinish     119
GarageCars         1
GarageArea         1
GarageQual       119
GarageCond       119
PoolQC          2037
Fence           1633
MiscFeature     1974
dtype: int64

In [16]:
# After the transformation the rows with NA are deleted form the dataframe

train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)

# Number of observations with NA after the transformation

train_t[imputer.variables_].isna().sum()

MSZoning        0
LotFrontage     0
Alley           0
Utilities       0
Exterior1st     0
Exterior2nd     0
MasVnrType      0
MasVnrArea      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinSF1      0
BsmtFinType2    0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
BsmtFullBath    0
BsmtHalfBath    0
Functional      0
FireplaceQu     0
GarageType      0
GarageYrBlt     0
GarageFinish    0
GarageCars      0
GarageArea      0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [17]:
# in this case, all observations will be dropped
# because all of them show NA at least in 1 variable

train_t.shape

(0, 79)

## Drop rows with % of missing data

Not to end up with an empty dataframe, let's drop rows that have less than 75% of the variables with values.

In [18]:
# Find variables with NA

imputer = DropMissingData(
    missing_only=True,
    threshold=0.75,
)

imputer.fit(X_train)

DropMissingData(threshold=0.75)

In [19]:
# After the transformation the rows with NA are deleted form the dataframe

train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)

train_t.shape

(2043, 79)